In [1]:
import time
import torch.backends.cudnn as cudnn
from torch import nn
from easydict import EasyDict as edict
from models import Generator, Discriminator, TruncatedVGG19
from datasets import SRDataset
from utils import *
from solver import train

In [2]:
from solver import train_init

In [3]:
# config
config = edict()
config.csv_folder = '../data/SRDataset'
config.HR_data_folder = '../data/SRDataset/div2k/DIV2K_train_HR'
config.LR_data_folder = '../data/SRDataset/div2k/DIV2K_train_LR_bicubic/X4'
config.crop_size = 96
config.scaling_factor = 4

# Generator parameters
config.G = edict()
config.G.large_kernel_size = 9
config.G.small_kernel_size = 3
config.G.n_channels = 64
config.G.n_blocks = 16

# Discriminator parameters
config.D = edict()
config.D.kernel_size = 3
config.D.n_channels = 64
config.D.n_blocks = 8
config.D.fc_size = 1024

# Learning parameters
config.checkpoint_init = None
config.checkpoint = None # path to model (SRGAN) checkpoint, None if none
config.batch_size = 64
config.start_epoch = 0
config.start_epoch_init = 0
config.epochs = 2
config.workers = 4
config.vgg19_i = 5  # the index i in the definition for VGG loss; see paper
config.vgg19_j = 4  # the index j in the definition for VGG loss; see paper
config.beta = 1e-3  # the coefficient to weight the adversarial loss in the perceptual loss
config.print_freq = 100
config.lr = 1e-3
config.SGD = edict()
config.SGD.lr= 5e-2
config.SGD.momentum=0.9

# Default device
config.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
config.device = "mps"
cudnn.benchmark = True

In [4]:
if config.checkpoint_init is None and config.checkpoint is None:
    # Generator
    generator = Generator(config)
    # generator's optimizer in initialization phase
    optimizer_g_init = torch.optim.SGD(params=filter(lambda p: p.requires_grad, generator.parameters()), 
                                       lr=config.SGD.lr, 
                                       momentum=config.SGD.momentum)
    # generator's optimizer
    optimizer_g = torch.optim.Adam(params=filter(lambda p: p.requires_grad, generator.parameters()),
                                   lr=config.lr)

    # Discriminator
    discriminator = Discriminator(config)
    optimizer_d = torch.optim.Adam(params=filter(lambda p: p.requires_grad, discriminator.parameters()),
                                   lr=config.lr)

elif config.checkpoint_init is not None and config.checkpoint is None:
    checkpoint = torch.load(config.checkpoint_init)
    config.start_epoch_init = checkpoint['epoch'] + 1
    generator = checkpoint['model']
    optimizer_g_init = checkpoint['optimizer']
    print("\nLoaded checkpoint from epoch %d.\n" % (checkpoint['epoch'] + 1))

    # generator's optimizer
    optimizer_g = torch.optim.Adam(params=filter(lambda p: p.requires_grad, generator.parameters()),
                                   lr=config.lr)

    # Discriminator
    discriminator = Discriminator(config)
    optimizer_d = torch.optim.Adam(params=filter(lambda p: p.requires_grad, discriminator.parameters()),
                                   lr=config.lr)


elif config.checkpoint is not None:
    checkpoint = torch.load(config.checkpoint)
    config.start_epoch = checkpoint['epoch'] + 1
    generator = checkpoint['generator']
    discriminator = checkpoint['discriminator']
    optimizer_g = checkpoint['optimizer_g']
    optimizer_d = checkpoint['optimizer_d']
    print("\nLoaded checkpoint from epoch %d.\n" % (checkpoint['epoch'] + 1))

In [5]:
# Truncated VGG19 network to be used in the loss calculation
truncated_vgg19 = TruncatedVGG19(i=config.vgg19_i, j=config.vgg19_j)
truncated_vgg19.eval()

/Users/chaoxiong.yang/opt/anaconda3/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/chaoxiong.yang/opt/anaconda3/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


TruncatedVGG19(
  (model): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 

In [6]:
# Loss functions
init_loss_criterion = nn.MSELoss()
content_loss_criterion = nn.MSELoss()
adversarial_loss_criterion = nn.BCEWithLogitsLoss()

In [7]:
# Move to default device
generator = generator.to(config.device)
discriminator = discriminator.to(config.device)
truncated_vgg19 = truncated_vgg19.to(config.device)
content_loss_criterion = content_loss_criterion.to(config.device)
adversarial_loss_criterion = adversarial_loss_criterion.to(config.device)

In [8]:
# Custom dataloaders
train_dataset = SRDataset(split='train', config=config)
train_loader = torch.utils.data.DataLoader(train_dataset,
                                           batch_size=config.batch_size,
                                           shuffle=True, 
                                           num_workers=config.workers,
                                           pin_memory=True)

In [ ]:
# initialize learning (G)
config.n_epoch_init = 1

for epoch in range(config.start_epoch_init, config.n_epoch_init):
    train_init(train_loader=train_loader,
         model=generator,
         loss_criterion=init_loss_criterion,
         optimizer=optimizer_g_init,
         epoch=epoch,
         device=config.device,
         print_freq=config.print_freq
    )
    torch.save({'epoch': epoch,
              'model': generator,
              'optimizer': optimizer_g_init},
              'checkpoint_generator.pth.tar')

Epoch: [0][0/1250]----Batch Time 6.020 (6.020)----MSE Loss 22.5502 (22.5502)
Epoch: [0][100/1250]----Batch Time 2.301 (1.099)----MSE Loss nan (nan)


In [ ]:
# Epochs
for epoch in range(config.start_epoch, config.epochs):
    # At the halfway point, reduce learning rate to a tenth
    if epoch == int(config.epochs / 2 + 1):
        adjust_learning_rate(optimizer_g, 0.1)
        adjust_learning_rate(optimizer_d, 0.1)
    # One epoch's training
    train(train_loader=train_loader,
          generator=generator,
          discriminator=discriminator,
          truncated_vgg19=truncated_vgg19,
          content_loss_criterion=content_loss_criterion,
          adversarial_loss_criterion=adversarial_loss_criterion,
          optimizer_g=optimizer_g,
          optimizer_d=optimizer_d,
          epoch=epoch,
          device=config.device,
          beta=config.beta,
          print_freq=config.print_freq)
    # Save checkpoint
    torch.save({'epoch': epoch,
                'generator': generator,
                'discriminator': discriminator,
                'optimizer_g': optimizer_g,
                'optimizer_d': optimizer_d},
                'checkpoint_srgan2.pth.tar')